# **IMPORTING LIBRARIES**

In [1]:
import sqlite3

# **CONNECTING TO DATABASE**

In [2]:
conn = sqlite3.connect('ecommerce_star.db')
cursor = conn.cursor()

# **Q1 : Retrieve the top 5 customers who have made the highest average order amounts in the last 6 months. The average order amount should be calculated for each customer, and the result should be sorted in descending order.**

In [3]:
cursor.execute('''SELECT CustomerID, AVG(TotalAmount) AS AvgOrderAmount
FROM FactOrders
WHERE OrderDateID >= (SELECT MAX(DateID) FROM DimDate WHERE Date >= DATE('now', '-6 months'))
GROUP BY CustomerID
ORDER BY AvgOrderAmount DESC
LIMIT 5''');
cursor.fetchall()

[(1, 834.16)]

# **Q2 : Retrieve the list of customer whose order value is lower this year as compared to previous year**

In [4]:
cursor.execute('''WITH OrderValues AS (
    SELECT CustomerID,
           SUM(TotalAmount) AS TotalOrderAmount,
           STRFTIME('%Y', DATE) AS OrderYear
    FROM FactOrders
    INNER JOIN DimDate ON FactOrders.OrderDateID = DimDate.DateID
    GROUP BY CustomerID, OrderYear
)
SELECT o1.CustomerID
FROM OrderValues o1
INNER JOIN OrderValues o2 ON o1.CustomerID = o2.CustomerID AND o1.OrderYear = STRFTIME('%Y', DATE('now')) AND o2.OrderYear = CAST(STRFTIME('%Y', DATE('now')) - 1 AS TEXT)
WHERE o1.TotalOrderAmount < o2.TotalOrderAmount;
''');
cursor.fetchall()

[(3,), (4,), (7,), (10,)]

# **Q4 : Retrieve the list of top 5 selling products. Further bifurcate the sales by product variants**

In [5]:
cursor.execute('''WITH ProductSales AS (
    SELECT dp.ProductName,
           dpv.VariantName,
           SUM(fod.Quantity) AS TotalSales
    FROM FactOrderDetails fod
    INNER JOIN DimProductVariants dpv ON fod.ProductVariantID = dpv.ProductVariantID
    INNER JOIN DimProducts dp ON dpv.ProductID = dp.ProductID
    GROUP BY dp.ProductName, dpv.VariantName
)
SELECT ProductName,
       VariantName,
       TotalSales
FROM ProductSales
ORDER BY TotalSales DESC
LIMIT 5;
''')
cursor.fetchall()

[('Very', 'Very Radio', 443),
 ('Treat', 'Treat Act', 427),
 ('Writer', 'Writer Draw', 411),
 ('Region', 'Region Nice', 406),
 ('Develop', 'Develop Begin', 395)]